# Get water levels

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import requests

In [ ]:
fig, ax = plt.subplots()
ax.plot(df.index, df["RSWL"], '.-')
ax.set_title("WLG051")
ax.set_xlabel("Year")
ax.set_ylabel("RSWL")

In [6]:
base_url = "https://www.waterconnect.sa.gov.au/_layouts/15/dfw.sharepoint.wdd/WDDDMS.ashx/"

url = base_url + "GetGridData"
rest_params = {"Box": "-35.25,138.55,-35.20,138.6"}
response = requests.get(url, params=rest_params)

# Convert to DataFrame
data = response.json()
df = pd.json_normalize(data)

In [7]:
df2 = df.dropna(subset="OBSNUMBER")
df2 = df2.loc[df2["WATER"] == 'Y']

writer = pd.ExcelWriter("wl_data.xlsx")
for dhno in df2["DHNO"]:
    print(dhno)
    url = base_url + "GetWaterLevelDetails"
    rest_params = {"DHNO": dhno}

    response = requests.get(url, params=rest_params)
    data = response.json()

    dfwl = pd.json_normalize(data)
    dfwl.index = pd.to_datetime(dfwl["OBS_DATE"])

    dfwl.to_excel(writer, sheet_name=str(dhno))
    
writer.close()

In [ ]:


lat_long_coordinates = gpd.points_from_xy(df["LON"], df["LAT"])
gdf = gpd.GeoDataFrame(
    df, 
    geometry=lat_long_coordinates, 
    crs="EPSG:4326",
)

Path('shp').mkdir(exist_ok=True)
gdf.to_file("shp/borehole_data.shp")